In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
from os import path
import pickle

# Routines for calculating and plotting phase diagrams in the full Fo-SiO$_2$ binary system

import the thermocodegen reaction object

In [ ]:
import py_fo_sio2_poly_linear_rxns as pfs
rxn = pfs.fo_sio2_poly_linear_rxns()
rxn.report()

convenience indices for phases and components

In [ ]:
iLq = 0
iOl = 1
iOpx = 2
iSi = 3
kSi = 0
kFo = 1

### Some utility functions for converting between compositions and Temperature units

In [ ]:
# load liquid composition vector given just x_si204
set_xliq = lambda xq : np.array([xq,1.-xq])

# convert mole fractions between moles of Si2O4 and SiO2
si2_to_si = lambda xq : 2.*xq/(1.+xq)
si_to_si2 = lambda xq : xq/(2.-xq)

# convert temperatures
to_Kelvin = lambda T : T + 273.15
to_Celsius = lambda T : T - 273.15

### A routine to extract liquidus surfaces and critical points from the phase diagram

In [ ]:
def calc_phasediagram(P,rxn,Trange = (1000., 2300.), N=101):
    ''' extracts liquidus and univariant points from the fo-sio2 reactive system 
        using a combination of matplotlib contour to extract 0 contours of the affinity functions 
        as well as scipy.optimize.root to calculate invariant points
        
        Parameters:
        ------------
            P: float
                pressure in bars
            rxn: ThermoCodegen reaction object
            Trange: tuple of floats (optional)
                plotting range for Temperature
            N: int
                number of x,T intervals in contour plots
    
        Returns:
        --------
            dictionary containing liquidus (x,T) pairs for the three cotetics plus (x,T) values of 
            univariant points for 
                Fo-En-Lq equilibrium
                En-Si-Lq equilibrium
                Lq-Lq-Si equilbrium
    '''
    
    

    # Load the Liquid phase object
    iLq = 0
    Lq = rxn.phases()[iLq]
    
    # initialize Concentration matrix
    C = rxn.zero_C()
    for i in range(1,4):
        C[i] = [1.]
        
    # initialized dictionary
    _dict = {}
    
    # find the univariant points
    
    # Utility functions for calculating the affinities of reactions with correct input order
    def set_liq(c, C):
        C[iLq] = set_xliq(c)
        return C
    
    foenA = lambda u, P, C : rxn.A(u[0],P,set_liq(u[1],C))[0:2]
    ensiA = lambda u, P, C : rxn.A(u[0],P,set_liq(u[1],C))[1:3]
    

    # fo en univariant
    u0 = [2100.,0.4]
    sol = root(foenA,u0,(P,C))
    if sol.success:
        T,c = sol.x
        x = Lq.c_to_x(set_xliq(c))[kSi]
        _dict['fo_en_lq'] = [si2_to_si(x), to_Celsius(T)]
    else:
        print('FoEn Equilbrium failed: {}'.format(sol.message))
    
    # en qz univariant
    u0 = [2000.,0.5]
    sol = root(ensiA,u0,(P,C))
    if sol.success:
        T,c = sol.x
        x = Lq.c_to_x(set_xliq(c))[kSi]
        _dict['en_qz_lq'] = [si2_to_si(x), to_Celsius(T)]
    else:
        print('EnQz Equilbrium failed: {}'.format(sol.message))
    
    # Find liquidus surfaces using plt.contour
    
    # set  Temperature range in degrees C
    t = np.linspace(Trange[0],Trange[1],N)

    # mol fraction range for x_siO2 (with minor regularization)
    eps = 1.e-3
    x = np.linspace(eps,1.-eps,N)
    
    # create MeshGrids for X,T
    X,T = np.meshgrid(x,t)
    Xf = X.flatten()
    
    
    
    # create flattened arrays
    A0 = np.empty(T.flatten().shape)
    A1 = A0.copy()
    A2 = A0.copy()
    
    #calculate gridded Affinities for the three reactions
    for k, tk in enumerate(to_Kelvin(T.flatten())):
        C[iLq] = Lq.x_to_c(set_xliq(si_to_si2(Xf[k])))
        A0[k], A1[k], A2[k] = rxn.A(tk,P,C)
    
    # and reshape
    A0 = A0.reshape(T.shape)
    A1 = A1.reshape(T.shape)
    A2 = A2.reshape(T.shape)
    
    # extract A_j = 0 contours
    cs0 = plt.contour(x,t,A0,[0])
    cs1 =  plt.contour(x,t,A1,[0])
    cs2 =  plt.contour(x,t,A2,[0])
    plt.close()
    fo_liq = cs0.allsegs[0][0].T
    en_liq = cs1.allsegs[0][0].T
    si_liq = cs2.allsegs[0][0].T    
    
    _dict['fo_liq'] = fo_liq
    _dict['en_liq'] = en_liq
    _dict['si_liq'] = si_liq

    # find solvus point
    i_si_max = np.argmax(si_liq[1])
    tail = si_liq[:,i_si_max:]
    i_si_min = np.argmin(tail[1,:])
    _dict['si_solvus'] = tail[:,i_si_min]
    
    return _dict
        

### A routine to generate phase diagrams for the fo_sio2 system

In [ ]:
def plot_phasediagram(P, rxn, Trange=(1300,2200), N=101):
    ''' Generates plots for phase diagrams of the  fo-sio2 system 
        calls calc_phasediagram(P,rxn) to generate liquidus and critical point dictionary
        
        Parameters:
        -----------
        P: float
            pressure in bars
        rxn: ThermoCodegen reaction object
        Trange: tuple of floats (optional)
            plotting range for Temperature
        N: int
            number of x,T intervals in contour plots
            
        Returns:
        --------
        fig:  matplotlib figure
        ax:   current matplotlib axis
        model_dict:  dictionary of liquidus and critical points
    
    '''
    
    # calculate various cotectics and critical points of the phase diagram
    _dict = calc_phasediagram(P,rxn,N=N)
    
    # extract X_sio2 values of critical points
    x_0 = _dict['fo_en_lq'][0]
    x_1 = _dict['en_qz_lq'][0]
    x_2 = _dict['si_solvus'][0]
    
    
    
    
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(1,1,1)
    # plot Fo-lq cotectic
    imax = np.argmin(np.abs(_dict['fo_liq'][0] - x_0))
    ax.plot(_dict['fo_liq'][0,:imax],_dict['fo_liq'][1,:imax],'r')
    
    # plot En-lq cotectic
    imin =  np.argmin(np.abs(_dict['en_liq'][0] - x_0))
    imax = np.argmin(np.abs(_dict['en_liq'][0] - x_1))
    ax.plot(_dict['en_liq'][0,imin:imax],_dict['en_liq'][1,imin:imax],'g')
    
    #plot Si-lq cotectic (with solvus)
    imin = np.argmin(np.abs(_dict['si_liq'] - x_1))
    ax.plot(_dict['si_liq'][0,imin:],_dict['si_liq'][1,imin: ],'b')

    # now put in all the fun horizontal lines
    ax.plot([0.,max(x_0,0.5)], _dict['fo_en_lq'][1]*np.ones(2),'k')
    ax.plot([0.5,1.0], _dict['en_qz_lq'][1]*np.ones(2),'k')
    ax.plot([0.5,0.5],[Trange[0], _dict['fo_en_lq'][1]],'k')
    
    imin = np.argwhere(_dict['si_liq'][1] >= _dict['si_solvus'][1]).min()
    ax.plot([_dict['si_liq'][0,imin], 1.], _dict['si_solvus'][1]*np.ones(2),'k')
    ax.grid()
    ax.set_xlabel('SiO$_2$ (mol fraction)')
    ax.set_ylabel('$T$ (C)')
    ax.set_ylim(Trange)
    ax.set_xlim([0.,1.])
    ax.set_title('$P$ = {} bar'.format(P),fontsize=18)

    return fig, ax, _dict

### Compare phase diagrams at 1 bar and 10 kbar

In [ ]:
fig, ax, model_dict = plot_phasediagram(1.,rxn, Trange=(1300,2200),N=200)
ax.text(0.2,1400,'Fo + En',horizontalalignment='center',fontsize=14)
ax.text(0.2,1650,'Fo + Lq',horizontalalignment='center',fontsize=14)
ax.text(0.8,1400,'En + Qz',horizontalalignment='center',fontsize=14)
ax.text(0.8,1550,'Qz + Lq',horizontalalignment='center',fontsize=14)
ax.text(0.8,1650,' 2 Lq',horizontalalignment='center',fontsize=14)
ax.text(0.5,1850,' Lq',horizontalalignment='center',fontsize=14)
plt.savefig('Fo-sio2-phase_diagram_1bar.png')
plt.show()

In [ ]:
fig, ax, model_dict = plot_phasediagram(10000.,rxn, Trange=(1300,2200),N=200)
ax.text(0.2,1500,'Fo + En',horizontalalignment='center',fontsize=14)
ax.text(0.2,1750,'Fo + Lq',horizontalalignment='center',fontsize=14)
ax.text(0.8,1500,'En + Qz',horizontalalignment='center',fontsize=14)
ax.text(0.8,1800,'Qz + Lq',horizontalalignment='center',fontsize=14)
ax.text(0.8,2000,' 2 Lq',horizontalalignment='center',fontsize=14)
ax.text(0.5,1850,' Lq',horizontalalignment='center',fontsize=14)
plt.savefig('Fo-sio2-phase_diagram_10kbar.png')
plt.show()